In [1]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [2]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   0%|          | 98.3k/84.1M [00:00<01:33, 900kB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 42.5MB/s]


#### The code to reverse the reviews

In [74]:
for i in range(len(train_data)):
  vars(train_data.examples[i]).get("text").reverse()

In [75]:
for i in range(len(test_data)):
  vars(test_data.examples[i]).get("text").reverse()

In [76]:
print(vars(train_data.examples[0]))


{'text': ['This', 'movie', ',', 'supposedly', 'a', 'thriller', ',', 'had', 'about', 'five', 'sub', '-', 'plots', 'that', 'developed', 'simultaneously', 'to', 'climax', 'at', 'the', 'end', ',', 'but', 'it', 'ended', 'up', 'more', 'as', 'a', 'yawn.<br', '/><br', '/>The', 'writing', 'was', 'trite', ',', 'the', 'pace', 'was', 'slow', 'and', 'disjointed', 'and', 'the', 'characters', 'were', 'boring', '.', 'George', 'Clooney', 'looks', 'like', 'he', 'needs', 'to', 'get', 'into', 'a', 'gym', ',', 'Matt', 'Damon', 'hammed', 'it', 'up', 'and', 'the', 'others', 'seemed', 'to', 'read', 'their', 'lines', 'from', 'a', 'teleprompter.<br', '/><br', '/>The', 'worst', 'part', 'of', 'this', 'movie', 'was', 'that', 'it', 'was', 'carefully', 'crafted', 'to', 'be', 'politically', 'correct', 'and', 'so', 'it', 'ended', 'up', 'saying', 'nothing', 'at', 'all', 'about', 'big', 'business', ',', 'oil', 'and', 'the', 'Middle', '-', 'East', '.', 'I', 'have', 'seen', 'documentaries', 'provide', 'more', 'excitement'

In [77]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [78]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [79]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

In [183]:

import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        self.rnn_2 = nn.LSTM(hidden_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        # text = torch.fliplr(text)
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu())
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        packed_embedded = hidden
        for i in range(2):

          packed_output, (hidden, cell) = self.rnn_2(packed_embedded)
          packed_embedded = hidden
        

        #unpack sequence
        # output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

In [184]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 1
BIDIRECTIONAL = True
DROPOUT = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [185]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,286,569 trainable parameters


In [186]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [187]:
model.embedding.weight.data.copy_(pretrained_embeddings)


tensor([[-1.4488,  2.1031,  0.3032,  ..., -1.2534,  0.0336,  0.6337],
        [ 0.0866, -0.6833, -0.0421,  ...,  0.6889,  1.2435,  1.7282],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.8108, -1.5292,  0.2283,  ...,  0.4888,  0.3059,  1.2459],
        [-1.0339, -0.7886, -0.1213,  ...,  1.3091,  0.4372,  1.5440],
        [ 2.1644, -0.0050,  0.4936,  ...,  0.5863, -0.7850, -1.0199]])

In [188]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.8108, -1.5292,  0.2283,  ...,  0.4888,  0.3059,  1.2459],
        [-1.0339, -0.7886, -0.1213,  ...,  1.3091,  0.4372,  1.5440],
        [ 2.1644, -0.0050,  0.4936,  ...,  0.5863, -0.7850, -1.0199]])


In [189]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [190]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [191]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [192]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [193]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [194]:

import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [195]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 10s
	Train Loss: 0.662 | Train Acc: 60.55%
	 Val. Loss: 0.647 |  Val. Acc: 71.88%
Epoch: 02 | Epoch Time: 0m 10s
	Train Loss: 0.627 | Train Acc: 63.44%
	 Val. Loss: 0.687 |  Val. Acc: 52.41%
Epoch: 03 | Epoch Time: 0m 10s
	Train Loss: 0.685 | Train Acc: 53.48%
	 Val. Loss: 0.683 |  Val. Acc: 58.90%
Epoch: 04 | Epoch Time: 0m 10s
	Train Loss: 0.650 | Train Acc: 61.77%
	 Val. Loss: 0.533 |  Val. Acc: 75.02%
Epoch: 05 | Epoch Time: 0m 10s
	Train Loss: 0.470 | Train Acc: 78.51%
	 Val. Loss: 0.360 |  Val. Acc: 84.77%
Epoch: 06 | Epoch Time: 0m 10s
	Train Loss: 0.303 | Train Acc: 87.90%
	 Val. Loss: 0.316 |  Val. Acc: 86.86%
Epoch: 07 | Epoch Time: 0m 10s
	Train Loss: 0.209 | Train Acc: 91.98%
	 Val. Loss: 0.367 |  Val. Acc: 86.33%
Epoch: 08 | Epoch Time: 0m 10s
	Train Loss: 0.155 | Train Acc: 94.51%
	 Val. Loss: 0.350 |  Val. Acc: 88.01%
Epoch: 09 | Epoch Time: 0m 10s
	Train Loss: 0.108 | Train Acc: 96.29%
	 Val. Loss: 0.366 |  Val. Acc: 87.10%
Epoch: 10 | Epoch T